Importing the login function from the Hugging Face Hub library.
Logging into the Hugging Face Hub.

In [ ]:
from huggingface_hub import login
login()

Download or upgrades the transformers, datasets, and accelerate packages.

In [ ]:
pip install -U transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


Load tokenizer for Gemma 2B-it model and Gemma 2B-it model for causal language modeling

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

helper functions for run_test function

In [ ]:
#convert 0123 to ABCD
def toABCD(num):
  abcd = "ABCD"
  return abcd[num]

#convert ABCD to 0123
def to1234(c):
  return ord(c) - ord("A")

#the first occurrence of any letter from "ABCD" found in the input text, or "E" if none are found
def getABCD(text):
  ABCD = "ABCD"
  ret = "E"
  for c in text:
    if ret!="E":
      break
    for i in ABCD:
      if c==i:
        ret = i
        break
  return ret

#Generate a multiple-choice prompt for a question with four choices, optionally including the correct answer if specified.
def write_prompt(args, isShot):
  prompt = args['question'] + '\n'
  for i in range(4):
    prompt += (toABCD(i) + ". " + args['choices'][i] + '\n')
  prompt += "Answer:"
  if isShot:
    prompt += toABCD(args['answer']) + ". " + args['choices'][args['answer']] + '\n\n'
  return prompt

# Prompt Generation Function
# Generates a prompt
def prompt(num, dataset, dataset_test_i):
  prompt = "The following are multiple choice questions (with answers) about " + dataset['examples'][0]['subject'] + '.\n\n'
  for i in range(num):
    prompt += write_prompt(dataset['examples'][i], True)
  prompt += write_prompt(dataset_test_i, False)
  return prompt

In [ ]:
import time, re, json, array
from datasets import concatenate_datasets, load_dataset
from google.colab import drive

#load necessary files and data
drive.mount('/content/drive')
f = open('/content/drive/MyDrive/Colab Notebooks/CS 200/data.json')
file_path = '/content/drive/MyDrive/Colab Notebooks/CS 200/data.json'
data = json.load(f)

#run the test for a given subject with given number of shots.
def run_test(num, subject):
  #parameter initialization
  start_time = time.time()
  count = 0
  #create section to be used as examples
  dataset['examples'] = concatenate_datasets([dataset['dev'], dataset['validation']])
  for dataset_test_i in dataset['test']:
    #end the run_test function if all questions have been answered
    if count > 99:
      break
    #skip the question if it has already been answered
    if data[subject][num]["response"][count] > -1:
      count += 1
      continue
    #copy the answer from the test dataset to the data
    data[subject][num]["answer"][count] = dataset["test"]["answer"][count]
    #generate the prompt
    text = prompt(num, dataset, dataset_test_i)
    #tokenize the prompt
    encoding = tokenizer(text, return_tensors="pt")
    #generate the response
    outputs = model.generate(**encoding, max_length=(len(encoding["input_ids"][0])+4))
    print(count, tokenizer.decode(outputs[0]))
    result_text = tokenizer.decode(outputs[0])
    # Retrieve option, model chose from the output
    matches = re.findall(r'Answer:([A-D])\.', result_text)
    # Retrieve option, model chose from the output but when there's no example given
    if num == 0:
      matches = re.findall(r'Answer: ([A-D])\.', result_text)
    # Retrieve option, model chose from the output
    last_match = matches[-1] if matches else None
    if last_match:
      answer = last_match
    else:
      answer = "E"
    print(to1234(answer))
    #save the response from the model to the data
    data[subject][num]["response"][count] = to1234(answer)
    #save the data to the file
    with open(file_path, 'w') as json_file:
      json.dump(data, json_file)
      print("Updated data")
    count += 1
  print('Time it took', time.time() - start_time)

Mounted at /content/drive


Initialize variable to store responses from the model and answer.

In [ ]:
# all subjects
subjects = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']
# initialize array that stores all the responses and answers
responseArray = [-1] * 100
answerArray = [-1] * 100
newArray = {"num":-1, "answer":answerArray, "response":responseArray}
# initialize data, if subject is not in data, add it
for subject in subjects:
  if subject in data:
    print(subject)
  else:
    data[subject] = []
    for i in range(11):
      newArray["num"] = i
      data[subject].append(newArray.copy())

abstract_algebra
anatomy
astronomy
business_ethics
clinical_knowledge
college_biology
college_chemistry
college_computer_science
college_mathematics
college_medicine
college_physics
computer_security
conceptual_physics
econometrics
electrical_engineering
elementary_mathematics
formal_logic
global_facts
high_school_biology
high_school_chemistry
high_school_computer_science
high_school_european_history
high_school_geography
high_school_government_and_politics
high_school_macroeconomics
high_school_mathematics
high_school_microeconomics
high_school_physics
high_school_psychology
high_school_statistics
high_school_us_history
high_school_world_history
human_aging
human_sexuality
international_law
jurisprudence
logical_fallacies
machine_learning
management
marketing
medical_genetics
miscellaneous
moral_disputes
moral_scenarios
nutrition
philosophy
prehistory
professional_accounting
professional_law
professional_medicine
professional_psychology
public_relations
security_studies
sociology
us_f

Run test for all subjects

In [ ]:
# all subjects
subjects = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']
# subjects that already completed the test
subjects_completed = ['abstract_algebra', 'business_ethics', 'econometrics', 'formal_logic', 'us_foreign_policy', 'college_chemistry']
# subjects that could not complete the test due to the size of the prompt
subjects_pass = ['high_school_european_history', 'high_school_us_history', 'high_school_world_history']
#load necessary files and data
f = open('/content/drive/MyDrive/Colab Notebooks/CS 200/data.json')
file_path = '/content/drive/MyDrive/Colab Notebooks/CS 200/data.json'
data = json.load(f)
#run the test for all subjects
for subject in subjects:
  if subject in subjects_completed:
    continue
  if subject in subjects_pass:
    continue
  dataset = load_dataset("cais/mmlu", subject)
  for i in range(11):
    run_test(i, subject)

Generating test split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Time it took 0.03034663200378418
Time it took 0.023811817169189453
Time it took 0.018985271453857422
Time it took 0.02357316017150879
Time it took 0.022420883178710938
Time it took 0.01947331428527832
Time it took 0.02064990997314453
Time it took 0.019301652908325195
Time it took 0.019931316375732422
Time it took 0.021813631057739258
